## Hybrid Recommender System

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Importing the DataSet and viewing it

##### movies_metadata

In [2]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate

## Importing the dataSet and checking it
meta  = pd.read_csv(r'C:\Users\lenovo\Desktop\3\DataSet\movies_metadata.csv')
meta.head()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_51900\676985758.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  meta  = pd.read_csv(r'C:\Users\lenovo\Desktop\3\DataSet\movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##### ratings

In [3]:
# The subset of 100,000 ratings from 700 users on 9,000 movies
ratings = pd.read_csv(r'C:\Users\lenovo\Desktop\3\DataSet 2\ratings.csv')
ratings.head() # Movies in this dataset are rated out of 5 instead of 10

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


##### links

In [4]:
# TMDb and IMDb IDs of a small subset of 9,000 movies of the Full Dataset
links = pd.read_csv(r'C:\Users\lenovo\Desktop\3\DataSet 2\links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


##### keywords

In [5]:
# Movie plot keywords for the MovieLens movies
keywords = pd.read_csv(r'C:\Users\lenovo\Desktop\3\Dataset\keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


##### credits

In [6]:
# Cast and Crew Information for all movies in the dataset
credits = pd.read_csv(r'C:\Users\lenovo\Desktop\3\Dataset\credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [7]:
# -- Content-focused recommender --

meta['overview'] = meta['overview'].fillna('')
meta['overview'].head() # Sample descriptions

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

#### Checking the dataType of each file imported

In [8]:
# Check the datatype of "id" in movies_metadata.csv
pd.DataFrame({'feature':meta.dtypes.index, 'dtype':meta.dtypes.values})

,feature,dtype
0,adult,object
1,belongs_to_collection,object
2,budget,object
3,genres,object
4,homepage,object
5,id,object
6,imdb_id,object
7,original_language,object
8,original_title,object
9,overview,object


In [9]:
meta = meta.drop([19730, 29503, 35587]) # Remove these ids to solve ValueError: "Unable to parse string..."

# Convert object to int64 for compatibility during merging
meta['id'] = pd.to_numeric(meta['id'])

# Run  the following code for converting more than one value to integer
# def convert_int(x):
#     try:
#         return int(x)
#     except:
#         return np.nan

In [10]:
# Check the datatype of "tmdbId" in links_small.csv
pd.DataFrame({'feature':links.dtypes.index, 'dtype':links.dtypes.values})

,feature,dtype
0,movieId,int64
1,imdbId,int64
2,tmdbId,float64


#### Converting the float values to int values

In [11]:
# Convert float64 to int64
col=np.array(links['tmdbId'], np.int64)
links['tmdbId']=col

In [12]:
# Merge the dataframes on column "tmdbId"
meta.rename(columns={'id':'tmdbId'}, inplace=True)
meta = pd.merge(meta,links,on='tmdbId')
meta.drop(['imdb_id'], axis=1, inplace=True)
meta.head()

# Alternatively, run the following code to reduce the size of movies_metadata.csv to match links.csv
# meta = meta[meta['tmdbId'].isin(links)]
# meta.shape

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041


#### Removing the Stopwords

In [13]:
# Remove stop words and use TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')
# Construct TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(meta['overview'])
tfidf_matrix.shape


(9555, 30455)

#### Cosine Similarity

In [14]:
# Compute cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Get corresponding indices of the movies
indices = pd.Series(meta.index, index=meta['original_title']).drop_duplicates()

#### Recommendation Function

In [15]:
# Recommendation function
def recommend(title, cosine_sim=cosine_sim):
    
    # Get the index of the movie that matches the title
    idx = indices[title]
    # Get the pairwise similarity scores of all movies with the given movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 15 most similar movies
    sim_scores = sim_scores[1:16]
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    # Remove low-rated movies or outliers
    for i in movie_indices:
        pop = meta.at[i,'vote_average']
        if pop<5 or pop>10:
            movie_indices.remove(i)

    # Return the most similar movies qualifying the 5.0 rating threshold
    return meta[['original_title','vote_average']].iloc[movie_indices]


#### Testing the Model

In [16]:

recommend('The Conjuring')

,original_title,vote_average
9286,The Conjuring 2,7.0
8961,Dark Places,5.7
2877,The Spiral Staircase,6.6
7791,Grave Encounters,6.2
5502,The Turning Point,6.0
348,Jason's Lyric,5.9
5867,The Amityville Horror,6.0
7984,Sinister,6.8
8095,Dark Skies,6.1
8873,Promoción Fantasma,5.7


In [17]:

# -- User-focused recommender --

reader = Reader() # Used to parse a file containing ratings
df = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) # Split the data into folds


<generator object KFold.split at 0x0000016C025FABA0>

In [18]:
# Use Single Value Decomposition (SVD) for cross-validation and fitting
svd = SVD()
cross_validate(svd, df, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)

In [19]:

# Check a random user's ratings
ratings[ratings['userId'] == 10]

,userId,movieId,rating,timestamp
1119,10,296,1.0,1455303387
1120,10,356,3.5,1455301685
1121,10,588,4.0,1455306173
1122,10,597,3.5,1455357645
1123,10,912,4.0,1455302254
...,...,...,...,...
1254,10,119145,1.0,1455302650
1255,10,129428,3.5,1455357384
1256,10,136020,5.0,1455302192
1257,10,137595,4.0,1455356898


In [20]:
# Read the smaller links file again
links_df = pd.read_csv(r'C:\Users\lenovo\Desktop\3\Dataset\links.csv')
col=np.array(links_df['tmdbId'], np.int64)
links_df['tmdbId']=col

# Merge movies_metadata.csv and links_small.csv files
links_df = links_df.merge(meta[['title', 'tmdbId']], on='tmdbId').set_index('title')
links_index = links_df.set_index('tmdbId') # For label indexing

## Source Code for  Hybrid Recommender System

In [21]:
# Recommendation function
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = links_df.loc[title]['tmdbId'] # Get the corresponding tmdb id
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of the 30 most similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_index.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(15) # Display top 15 similar movies


## Testing Hybrid Model

In [22]:
# Recommendations for user with id 1
hybrid(1, 'The Conjuring')

,Title,Vote Average,TMDb Id,Estimated Prediction
348,Jason's Lyric,5.9,22067,4.647813
6652,[REC],7.1,8329,4.610659
8873,Ghost Graduation,5.7,86541,4.506868
7596,Midnight in Paris,7.4,59436,4.491244
2877,The Spiral Staircase,6.6,27452,4.389341
1849,The Texas Chain Saw Massacre,7.1,30497,4.355356
9034,Cornered!,4.1,32285,4.322775
7979,The Tall Man,5.7,86597,4.317856
9015,Deathgasm,6.4,323373,4.295809
8088,Safe Haven,6.9,112949,4.294840


In [23]:

# Recommendations for user with id 30
hybrid(30, 'The Conjuring')


,Title,Vote Average,TMDb Id,Estimated Prediction
6652,[REC],7.1,8329,4.479941
7596,Midnight in Paris,7.4,59436,4.440785
8873,Ghost Graduation,5.7,86541,4.333898
348,Jason's Lyric,5.9,22067,4.283181
8088,Safe Haven,6.9,112949,4.264811
9015,Deathgasm,6.4,323373,4.262047
7984,Sinister,6.8,82507,4.250042
5686,Salem's Lot,5.5,36763,4.210935
2877,The Spiral Staircase,6.6,27452,4.199572
9301,Lights Out,6.3,345911,4.065913


In [24]:
# Recommendations for user with id 500
hybrid(500, 'The Conjuring')

,Title,Vote Average,TMDb Id,Estimated Prediction
7596,Midnight in Paris,7.4,59436,3.584645
6652,[REC],7.1,8329,3.435765
9015,Deathgasm,6.4,323373,3.319245
5133,Night of the Living Dead,6.7,19185,3.289911
5502,The Turning Point,6.0,61280,3.226755
8088,Safe Haven,6.9,112949,3.219533
8961,Dark Places,5.7,182560,3.193568
9301,Lights Out,6.3,345911,3.187493
9286,The Conjuring 2,7.0,259693,3.183804
5850,Electra Glide in Blue,6.6,26332,3.182935


                                - - - - - - - - X X X X X X X X - - - - - - - -